In [1]:
from transformers import AutoTokenizer, BertModel
import torch
import numpy as np
import datasets
import pandas as pd
from tqdm import tqdm

In [7]:
sentence = 'I love you [SEP] I hate you'
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenized = tokenizer.encode_plus(sentence, return_tensors='pt', padding=True, truncation=True,add_special_tokens=True)
print(tokenized)
#decode the tokenized sentence
decoded = tokenizer.decode(tokenized['input_ids'][0])
print(decoded)

{'input_ids': tensor([[ 101, 1045, 2293, 2017,  102, 1045, 5223, 2017,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}
[CLS] i love you [SEP] i hate you [SEP]


In [ ]:
class Data(torch.utils.data.Dataset):
    def __init__(self, path):
        df = pd.read_csv(path)
        df['training'] = df.apply(lambda x: x['sentence1'] + ' [SEP] ' + x['sentence2'], axis=1)
        df['tokenized'] = df['training'].apply(lambda x: tokenizer.encode_plus(x, return_tensors='pt', padding=True, truncation=True,add_special_tokens=True))
        df['input_ids'] = df['tokenized'].apply(lambda x: x['input_ids'][0])
        df['attention_mask'] = df['tokenized'].apply(lambda x: x['attention_mask'][0])
        df['label'] = df['score'].apply(lambda x: torch.tensor(x))
        self.df = df

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        return self.df['input_ids'][idx], self.df['attention_mask'][idx], self.df['label'][idx]

def collate_fn(batch):
    input_ids = torch.nn.utils.rnn.pad_sequence([x[0] for x in batch])
    attention_mask = torch.nn.utils.rnn.pad_sequence([x[1] for x in batch])
    label = torch.stack([x[2] for x in batch])
    return input_ids, attention_mask, label

In [ ]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.fc1 = torch.nn.Linear(768, 40)
        self.fc2 = torch.nn.Linear(40, 1)

    def forward(self, input_ids, attention_mask):
        x = self.bert(input_ids, attention_mask)
        x = x['last_hidden_state'][:,0,:] #CLS token
        x = torch.nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
def train(model=None, train_dataloader=None,validation_dataloader=None,epochs=10,loss_fn=torch.nn.functional.mse_loss,
          lr=1e-3,batch_size=16):
    if train_dataloader is None:
        train_dataset = Data('Data/train.csv')
        train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    if validation_dataloader is None:
        validation_dataset = Data('Data/dev.csv')
        validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for i in range(epochs):
        print('Training for epoch',i+1)
        total_loss = 0
        for input_ids, attention_mask, label in tqdm(train_dataloader):
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            label = label.to(device)
            optimizer.zero_grad()
            output = model(input_ids, attention_mask)
            loss = loss_fn(output, label)
            loss.backward()
            optimizer.step()
            with torch.no_grad():
                total_loss += loss.item()
        print('Training loss',total_loss/len(train_dataloader))
        total_loss = 0
        print('Validation for epoch',i+1)
        with torch.no_grad():
            total_loss = 0
            for input_ids, attention_mask, label in tqdm(validation_dataloader):
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                label = label.to(device)
                output = model(input_ids, attention_mask)
                loss = loss_fn(output, label)
                total_loss += loss.item()
            print('Validation loss',total_loss/len(validation_dataloader))